In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pytimekr darts

In [ ]:
#Regression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
'''
from sklearn.model_selection import KFold, cross_val_score, train_test_split, TimeSeriesSplit
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, VotingRegressor, StackingRegressor, AdaBoostRegressor, HistGradientBoostingRegressor
from sklearn.dummy import DummyRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, f1_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder,OrdinalEncoder, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.inspection import PartialDependenceDisplay
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
'''

In [ ]:
from darts import TimeSeries
from darts.models import DLinearModel, Prophet, ARIMA
from darts.utils.missing_values import fill_missing_values

In [ ]:
import os
import random

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [ ]:
DATA_PATH = '/content/drive/MyDrive/crop_project/open'

In [ ]:
train = pd.read_csv(DATA_PATH + "/train.csv")
test = pd.read_csv(DATA_PATH + "/test.csv",index_col='timestamp', parse_dates=True)
ss = pd.read_csv(DATA_PATH + "/sample_submission.csv")
it = pd.read_csv(DATA_PATH + "/international_trade.csv")

In [ ]:
darts_data = train.rename(columns={'timestamp':'ds', 'price(원/kg)':'y'})
darts_data = darts_data[['ID','ds','y']]
darts_data['ID'] = darts_data['ID'].str.replace(r'_\d{8}$','', regex=True)

In [ ]:
from darts.models import ARIMA
from darts import TimeSeries
import itertools
import numpy as np
from sklearn.metrics import mean_squared_error

def arima_grid_search(df, p_values, d_values, q_values):
    best_score, best_cfg = float("inf"), None

    for p, d, q in itertools.product(p_values, d_values, q_values):
        scores = []

        for code in df['ID'].unique():
            data = df[df['ID'] == code].reset_index().drop(['ID'], axis=1).sort_values('ds')

            # TimeSeries로 변환
            ts = TimeSeries.from_dataframe(data, 'ds', 'y')

            # ARIMA 모델 초기화
            model = ARIMA(p=p, d=d, q=q)

            # ARIMA 모델 학습
            model.fit(ts)

            # 예측 수행
            forecast = model.predict(n=28)

            # 결과값 추출
            pred_y = forecast.values()
            true_y = ts.values()

            # 예측 성능 평가 (이 예시에서는 MSE 사용)
            mse = mean_squared_error(true_y[-28:], pred_y)

            scores.append(mse)

        # 현재 조합의 성능 평균 계산
        mean_score = np.mean(scores)

        # 현재 조합이 더 나은 경우 최적값 갱신
        if mean_score < best_score:
            best_score, best_cfg = mean_score, (p, d, q)

    return best_cfg

# 예제에서는 각 파라미터에 대해 0부터 2까지의 값 시도
p_values = range(5)
d_values = range(5)
q_values = range(5)

best_cfg = arima_grid_search(darts_data, p_values, d_values, q_values)
print("Best ARIMA Configuration:", best_cfg)


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood op

LinAlgError: ignored

In [ ]:
from prophet import Prophet

In [ ]:
def prophet_train(df):
    pred_list = []
    for code in darts_data['ID'].unique():
        d = darts_data[darts_data['ID'] == code].reset_index().drop(['ID'], axis=1).sort_values('ds')
        model = Prophet(growth = 'linear',
                        seasonality_mode = 'multiplicative',
                        yearly_seasonality = True,
                        weekly_seasonality = True,
                        daily_seasonality = True,
                        holidays = None,
                        changepoint_prior_scale = 0.05
                       )
        model.fit(d)
        future = pd.DataFrame()
        future['ds'] = pd.date_range(start='2023-03-04', periods=28, freq='D')
        forecast = model.predict(future)
        pred_y = forecast['yhat'].values
        pred_code = [str(code)] * len(pred_y)
        for y_val, id_val in zip(pred_y, pred_code):
            pred_list.append({'ID': id_val, 'y': y_val})
    pred = pd.DataFrame(pred_list)
    return pred
prophet_pred = prophet_train(darts_data)

DEBUG:cmdstanpy:input tempfile: /tmp/tmp2_xpa1fv/1brna7fq.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp2_xpa1fv/w3ut_xk9.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=52315', 'data', 'file=/tmp/tmp2_xpa1fv/1brna7fq.json', 'init=/tmp/tmp2_xpa1fv/w3ut_xk9.json', 'output', 'file=/tmp/tmp2_xpa1fv/prophet_modelojew59pd/prophet_model-20231117070439.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
07:04:39 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:04:40 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp2_xpa1fv/89894rn5.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp2_xpa1fv/bx179dko.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

In [ ]:
def DLinear_train(df):
    pred_list = []
    for code in darts_data['ID'].unique():
        d = darts_data[darts_data['ID'] == code].reset_index().drop(['ID'], axis=1).sort_values('ds')

        ts = TimeSeries.from_dataframe(d[['y']].astype('float32'))

        model = DLinearModel(input_chunk_length=128, output_chunk_length=28, batch_size=256, n_epochs=100)
        model.fit(ts)

        future = pd.DataFrame()
        future['ds'] = pd.date_range(start='2023-03-04',periods=28,freq='D') #prophet_data['ID'].nunique() = 39
        forecast = model.predict(len(future['ds']))
        pred_y = forecast.values()
        pred_code = [str(code)] * len(pred_y)
        for y_val, id_val in zip(pred_y, pred_code):
            pred_list.append({'ID':id_val, 'y':y_val})

    pred = pd.DataFrame(pred_list)

    return pred

DLinear_pred = DLinear_train(darts_data)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type             | Params
-----------------------------------------------------
0 | criterion       | MSELoss          | 0     
1 | train_metrics   | MetricCollection | 0     
2 | val_metrics     | MetricCollection | 0     
3 | decomposition   | _SeriesDecomp    | 0     
4 | linear_seasonal | Linear           | 3.6 K 
5 | linear_trend    | Linear           | 3.6 K 
-----------------------------------------------------
7.2 K     Trainable params
0         Non-trainable params
7.2 K     Total params
0.029     Total esti

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type             | Params
-----------------------------------------------------
0 | criterion       | MSELoss          | 0     
1 | train_metrics   | MetricCollection | 0     
2 | val_metrics     | MetricCollection | 0     
3 | decomposition   | _SeriesDecomp    | 0     
4 | linear_seasonal | Linear           | 3.6 K 
5 | linear_trend    | Linear           | 3.6 K 
-----------------------------------------------------
7.2 K     Trainable params
0         Non-trainable params
7.2 K     Total params
0.029     Total esti

...

Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type             | Params
-----------------------------------------------------
0 | criterion       | MSELoss          | 0     
1 | train_metrics   | MetricCollection | 0     
2 | val_metrics     | MetricCollection | 0     
3 | decomposition   | _SeriesDecomp    | 0     
4 | linear_seasonal | Linear           | 3.6 K 
5 | linear_trend    | Linear           | 3.6 K 
-----------------------------------------------------
7.2 K     Trainable params
0         Non-trainable params
7.2 K     Total params
0.029     Total esti

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
from darts.models import ARIMA
from darts import TimeSeries

def ARIMA_train(df):
    pred_list = []

    for code in df['ID'].unique():
        d = df[df['ID'] == code].reset_index().drop(['ID'], axis=1).sort_values('ds')

        # TimeSeries로 변환
        ts = TimeSeries.from_dataframe(d, 'ds', 'y')

        # ARIMA 모델 초기화
        model = ARIMA(p=1, d=1, q=1)

        # ARIMA 모델 학습
        model.fit(ts)

        # 예측 수행
        forecast = model.predict(n=28)

        # 결과값 추출
        pred_y = forecast.values()
        pred_code = [str(code)] * len(pred_y)

        for y_val, id_val in zip(pred_y, pred_code):
            pred_list.append({'ID': id_val, 'y': y_val})

    pred = pd.DataFrame(pred_list)
    return pred

ARIMA_pred = ARIMA_train(darts_data)

In [ ]:
DLinear_pred

,ID,y
0,TG_A_J,[2831.0813]
1,TG_A_J,[1244.7177]
2,TG_A_J,[2728.9585]
3,TG_A_J,[2701.161]
4,TG_A_J,[2510.7556]
...,...,...
1087,RD_F_J,[388.6226]
1088,RD_F_J,[363.42368]
1089,RD_F_J,[380.87726]
1090,RD_F_J,[376.68332]


In [ ]:
prophet_pred

,ID,y
0,TG_A_J,3022.001364
1,TG_A_J,-247.030384
2,TG_A_J,3778.194959
3,TG_A_J,3917.751382
4,TG_A_J,3946.062384
...,...,...
1087,RD_F_J,311.147599
1088,RD_F_J,324.278876
1089,RD_F_J,307.985411
1090,RD_F_J,298.655903


In [ ]:
# ID와 예측값('y')만 남기고 나머지 제거
prophet_pred = prophet_pred[['ID', 'y']]
DLinear_pred = DLinear_pred[['ID', 'y']]
ARIMA_pred = ARIMA_pred[['ID', 'y']]

# ID를 기준으로 데이터 병합
merged_pred = pd.merge(prophet_pred, DLinear_pred, on='ID', suffixes=('_prophet', '_DLinear'))
merged_pred = pd.merge(merged_pred, ARIMA_pred, on='ID')

# 예측값 평균 계산
merged_pred['ensemble'] = (merged_pred['y_prophet'] + merged_pred['y_DLinear'] + merged_pred['y']) / 3.0

# 필요한 컬럼만 선택
final_pred = merged_pred[['ID', 'ensemble']]

In [ ]:
final_pred

,ID,ensemble
0,TG_A_J,[2777.1461283057324]
1,TG_A_J,[2761.4973869454934]
2,TG_A_J,[2760.864965201356]
3,TG_A_J,[2760.83940677124]
4,TG_A_J,[2760.838373863444]
...,...,...
856123,RD_F_J,[374.65065074219393]
856124,RD_F_J,[374.65065074219393]
856125,RD_F_J,[374.65065074219393]
856126,RD_F_J,[374.65065074219393]


In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/crop_project/open/sample_submission.csv')
sample_submission

,ID,answer
0,TG_A_J_20230304,0
1,TG_A_J_20230305,0
2,TG_A_J_20230306,0
3,TG_A_J_20230307,0
4,TG_A_J_20230308,0
...,...,...
1087,RD_F_J_20230327,0
1088,RD_F_J_20230328,0
1089,RD_F_J_20230329,0
1090,RD_F_J_20230330,0


In [ ]:
# 'sample_submission' 데이터프레임에서 'answer' 열의 값을 'final_pred'의 예측값으로 대체
sample_submission['answer'] = prophet_pred['y'].astype(float)

# 변경된 'sample_submission' 확인
print(sample_submission)

                   ID       answer
0     TG_A_J_20230304  3022.001364
1     TG_A_J_20230305  -247.030384
2     TG_A_J_20230306  3778.194959
3     TG_A_J_20230307  3917.751382
4     TG_A_J_20230308  3946.062384
...               ...          ...
1087  RD_F_J_20230327   311.147599
1088  RD_F_J_20230328   324.278876
1089  RD_F_J_20230329   307.985411
1090  RD_F_J_20230330   298.655903
1091  RD_F_J_20230331   296.572356

[1092 rows x 2 columns]


In [ ]:
sample_submission.to_csv('./prophet_submission.csv', index=False)